<a href="https://colab.research.google.com/gist/ruvnet/5cdbbd43ab3a0c728fdd3e7a2a8aedd9/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction: Multi-Agent Conversational System
#### [Created By rUv, cause he could.](https://github.com/ruvnet)

This notebook presents an advanced, modular multi-agent conversational system designed to navigate complex task trees within a financial system. The system leverages multiple specialized agents, each responsible for handling distinct tasks such as stock lookup, user authentication, account balance inquiries, money transfers, and overall orchestration of the conversation flow. The architecture is built to be highly customizable, allowing for the seamless integration of new agents and functionalities as needed.

#### Core Components:

1. **Stock Lookup Agent**:
   - Handles requests for stock price information.
   - Can assist users in searching for stock symbols based on company names and retrieving current trading prices.

2. **Authentication Agent**:
   - Manages user login processes by validating credentials and storing session tokens.
   - Ensures that sensitive operations, such as balance checks or money transfers, are only accessible to authenticated users.

3. **Account Balance Agent**:
   - Provides users with information on their account balances.
   - Requires authentication to ensure security and privacy.

4. **Money Transfer Agent**:
   - Facilitates secure transfers of funds between user accounts.
   - Verifies sufficient balance and user authentication before proceeding with transactions.

5. **Orchestration Agent**:
   - Acts as the central decision-making unit.
   - Determines which agent should handle the next step in the conversation based on the current user state and input.

#### Example Workflow:

- **User Interaction**: The user initiates the conversation by selecting an agent or inputting a query. The orchestration agent interprets the input and routes the request to the appropriate agent.
- **Task Execution**: The selected agent processes the request, whether it’s looking up a stock price, checking an account balance, or transferring funds.
- **State Management**: The system maintains the current state of the user, including session details and task progress, ensuring that each step of the conversation is informed by the user’s previous actions.
- **Dynamic Decision-Making**: Based on the outcomes of each interaction, the orchestration agent decides the next step, keeping the conversation focused and efficient.

### Summary

This notebook is a powerful tool for building and managing a multi-agent conversational system tailored to complex financial tasks. With its modular design, customizable features, and robust handling of interactions, it offers a flexible framework for creating sophisticated AI-driven financial assistants. Whether you’re managing simple inquiries or navigating intricate financial workflows, this system provides the tools and structure needed to develop a responsive and intelligent conversational experience.

## Install requirements and configure OpenAi API Key

In [ ]:
# Install the OpenAI library (uncomment if needed)
!pip install python-dotenv colorama llama-index

# Import necessary libraries
import openai
from google.colab import userdata

# Retrieve and set the API key
api_key = userdata.get('OPENAI_API_KEY')
openai.api_key = api_key

# Verify the API key is set (this is just for demonstration and should not be used in production code)
if openai.api_key:
    print("OpenAI API key is set. Ready to proceed!")
else:
    print("OpenAI API key is not set. Please check your setup.")

## Check everything is installed

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from enum import Enum
from typing import List
import pprint
from colorama import Fore, Back, Style
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent

class Speaker(str, Enum):
    STOCK_LOOKUP = "stock_lookup"
    AUTHENTICATE = "authenticate"
    ACCOUNT_BALANCE = "account_balance"
    TRANSFER_MONEY = "transfer_money"
    CONCIERGE = "concierge"
    ORCHESTRATOR = "orchestrator"

print("LlamaIndex import successful!") # Added a success message

LlamaIndex import successful!


## Multi Agent Configuration

A multi-agent conversational system for navigating a complex task tree.
In this demo, the user is navigating a financial system.

**Agent 1:** look up a stock price.
Does not require authentication, has specialized tools for looking up stock prices.

**Agent 2:** authenticate the user.
A required step before the user can interact with some of the other agents.

**Agent 3:** look up an account balance.
Requires the user to be authenticated first.

**Agent 4:** transfer money between accounts.
Requires the user to be authenticated first, and have looked up their balance already.

**Concierge agent:** a catch-all agent that helps navigate between the other 4.

**Orchestration agent:** decides which agent to run based on the current state of the user.

### Stock lookup agent
This section defines the Stock Lookup Agent, responsible for assisting users in finding stock prices. The agent can search for a stock symbol based on a company name and retrieve the current trading price of the stock. It includes tools for looking up stock prices, searching for stock symbols, and signaling the completion of the task. The agent is designed to guide users through the stock lookup process, ensuring accuracy by relying on predefined tools for symbol searches and enforcing the completion of tasks before transitioning to other agents.

In [ ]:
# Stock lookup agent
def stock_lookup_agent_factory(state: dict) -> OpenAIAgent:

    def lookup_stock_price(stock_symbol: str) -> str:
        """Useful for looking up a stock price."""
        print(f"Looking up stock price for {stock_symbol}")
        return f"Symbol {stock_symbol} is currently trading at $100.00"

    def search_for_stock_symbol(str: str) -> str:
        """Useful for searching for a stock symbol given a free-form company name."""
        print("Searching for stock symbol")
        return str.upper()

    def done() -> None:
        """When you have returned a stock price, call this tool."""
        print("Stock lookup is complete")
        state["current_speaker"] = None
        state["just_finished"] = True

    tools = [
        FunctionTool.from_defaults(fn=lookup_stock_price),
        FunctionTool.from_defaults(fn=search_for_stock_symbol),
        FunctionTool.from_defaults(fn=done),
    ]

    system_prompt = (f"""
        You are a helpful assistant that is looking up stock prices.
        The user may not know the stock symbol of the company they're interested in,
        so you can help them look it up by the name of the company.
        You can only look up stock symbols given to you by the search_for_stock_symbol tool, don't make them up. Trust the output of the search_for_stock_symbol tool even if it doesn't make sense to you.
        The current user state is:
        {pprint.pformat(state, indent=4)}
        Once you have supplied a stock price, you must call the tool "done" to signal that you are done.
        If the user asks to do anything other than look up a stock symbol or price, call the tool "done" to signal some other agent should help.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini"),
        system_prompt=system_prompt,
    )


### Auth Agent
This section defines the Authentication Agent, responsible for securely logging in users by storing their username and validating their password to generate a session token. The agent includes tools to record the username, handle the login process, check authentication status, and signal the completion of the authentication task. It guides users through the login process and ensures that only authenticated users can proceed with subsequent actions. Upon successful authentication, the agent concludes its task and hands off control to other agents if needed.

In [ ]:
# Auth Agent
def auth_agent_factory(state: dict) -> OpenAIAgent:

    def store_username(username: str) -> None:
        """Adds the username to the user state."""
        print("Recording username")
        state["username"] = username

    def login(password: str) -> None:
        """Given a password, logs in and stores a session token in the user state."""
        print(f"Logging in {state['username']}")
        # todo: actually check the password
        session_token = "output_of_login_function_goes_here"
        state["session_token"] = session_token

    def is_authenticated() -> bool:
        """Checks if the user has a session token."""
        print("Checking if authenticated")
        if state["session_token"] is not None:
            return True

    def done() -> None:
        """When you complete your task, call this tool."""
        print("Authentication is complete")
        state["current_speaker"] = None
        state["just_finished"] = True

    tools = [
        FunctionTool.from_defaults(fn=store_username),
        FunctionTool.from_defaults(fn=login),
        FunctionTool.from_defaults(fn=is_authenticated),
        FunctionTool.from_defaults(fn=done),
    ]

    system_prompt = (f"""
        You are a helpful assistant that is authenticating a user.
        Your task is to get a valid session token stored in the user state.
        To do this, the user must supply you with a username and a valid password. You can ask them to supply these.
        If the user supplies a username and password, call the tool "login" to log them in.
        The current user state is:
        {pprint.pformat(state, indent=4)}
        When you have authenticated, call the tool "done" to signal that you are done.
        If the user asks to do anything other than authenticate, call the tool "done" to signal some other agent should help.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini"),
        system_prompt=system_prompt,
    )


### Account balance agent
This section defines the Account Balance Agent, responsible for retrieving and providing account balances to authenticated users. The agent can assist users in looking up their account ID and then checking the balance of that account. It includes tools for verifying user authentication, retrieving account details, and completing the balance inquiry task. The agent is designed to ensure that users are authenticated before accessing balance information and to guide them through the process of checking their account balance. Once the task is completed, the agent hands off control to other agents if further assistance is required.

In [ ]:
# Account balance agent
def account_balance_agent_factory(state: dict) -> OpenAIAgent:

    def get_account_id(account_name: str) -> str:
        """Useful for looking up an account ID."""
        print(f"Looking up account ID for {account_name}")
        account_id = "1234567890"
        state["account_id"] = account_id
        return f"Account id is {account_id}"

    def get_account_balance(account_id: str) -> str:
        """Useful for looking up an account balance."""
        print(f"Looking up account balance for {account_id}")
        state["account_balance"] = 1000
        return f"Account {account_id} has a balance of ${state['account_balance']}"

    def is_authenticated() -> bool:
        """Checks if the user has a session token."""
        print("Account balance agent is checking if authenticated")
        if state["session_token"] is not None:
            return True

    def done() -> None:
        """When you complete your task, call this tool."""
        print("Account balance lookup is complete")
        state["current_speaker"] = None
        state["just_finished"] = True

    tools = [
        FunctionTool.from_defaults(fn=get_account_id),
        FunctionTool.from_defaults(fn=get_account_balance),
        FunctionTool.from_defaults(fn=is_authenticated),
        FunctionTool.from_defaults(fn=done),
    ]

    system_prompt = (f"""
        You are a helpful assistant that is looking up account balances.
        The user may not know the account ID of the account they're interested in,
        so you can help them look it up by the name of the account.
        The user can only do this if they are authenticated, which you can check with the is_authenticated tool.
        If they aren't authenticated, tell them to authenticate
        If they're trying to transfer money, they have to check their account balance first, which you can help with.
        The current user state is:
        {pprint.pformat(state, indent=4)}
        Once you have supplied an account balance, you must call the tool "done" to signal that you are done.
        If the user asks to do anything other than look up an account balance, call the tool "done" to signal some other agent should help.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini"),
        system_prompt=system_prompt,
    )


### Money Transfer Agent
This section defines the Money Transfer Agent, which is responsible for facilitating secure money transfers between user accounts. The agent ensures that the user is authenticated and that sufficient funds are available in the source account before proceeding with the transfer. It includes tools to verify account balance, confirm user authentication, and execute the transfer. The agent guides users through the money transfer process, ensuring all necessary conditions are met before completing the transaction. Once the transfer is successful, the agent signals completion, allowing other agents to take over if further assistance is needed.

In [ ]:
# Money Transfer Agent

def transfer_money_agent_factory(state: dict) -> OpenAIAgent:

    def transfer_money(from_account_id: str, to_account_id: str, amount: int) -> None:
        """Useful for transferring money between accounts."""
        print(f"Transferring {amount} from {from_account_id} account {to_account_id}")
        return f"Transferred {amount} to account {to_account_id}"

    def balance_sufficient(account_id: str, amount: int) -> bool:
        """Useful for checking if an account has enough money to transfer."""
        # todo: actually check they've selected the right account ID
        print("Checking if balance is sufficient")
        if state['account_balance'] >= amount:
            return True

    def has_balance() -> bool:
        """Useful for checking if an account has a balance."""
        print("Checking if account has a balance")
        if state["account_balance"] is not None:
            return True

    def is_authenticated() -> bool:
        """Checks if the user has a session token."""
        print("Transfer money agent is checking if authenticated")
        if state["session_token"] is not None:
            return True

    def done() -> None:
        """When you complete your task, call this tool."""
        print("Money transfer is complete")
        state["current_speaker"] = None
        state["just_finished"] = True

    tools = [
        FunctionTool.from_defaults(fn=transfer_money),
        FunctionTool.from_defaults(fn=balance_sufficient),
        FunctionTool.from_defaults(fn=has_balance),
        FunctionTool.from_defaults(fn=is_authenticated),
        FunctionTool.from_defaults(fn=done),
    ]

    system_prompt = (f"""
        You are a helpful assistant that transfers money between accounts.
        The user can only do this if they are authenticated, which you can check with the is_authenticated tool.
        If they aren't authenticated, tell them to authenticate first.
        The user must also have looked up their account balance already, which you can check with the has_balance tool.
        If they haven't already, tell them to look up their account balance first.
        The current user state is:
        {pprint.pformat(state, indent=4)}
        Once you have transferred the money, you can call the tool "done" to signal that you are done.
        If the user asks to do anything other than transfer money, call the tool "done" to signal some other agent should help.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini"),
        system_prompt=system_prompt,
    )

### Concierge agent
This section defines the Concierge Agent, which acts as a guide to help users navigate the financial system. The agent's primary role is to understand the user's needs by asking relevant questions and directing them to the appropriate actions. It can guide users through tasks such as looking up stock prices, authenticating their identity, checking account balances, and transferring money between accounts. The Concierge Agent uses a placeholder tool that performs no action but serves as a framework for interaction, ensuring that users are directed to the correct agent or task based on their requests and the current state of their session.

In [ ]:
# Concierge agent
def concierge_agent_factory(state: dict) -> OpenAIAgent:

    def dummy_tool() -> bool:
        """A tool that does nothing."""
        print("Doing nothing.")

    tools = [
        FunctionTool.from_defaults(fn=dummy_tool)
    ]

    system_prompt = (f"""
        You are a helpful assistant that is helping a user navigate a financial system.
        Your job is to ask the user questions to figure out what they want to do, and give them the available things they can do.
        That includes
        * looking up a stock price
        * authenticating the user
        * checking an account balance (requires authentication first)
        * transferring money between accounts (requires authentication and checking an account balance first)

        The current state of the user is:
        {pprint.pformat(state, indent=4)}
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini"),
        system_prompt=system_prompt,
    )

### Continuation agent
This section defines the Continuation Agent, which is designed to assess the current state of the user and determine the next steps in their interaction with the financial system. The agent's role is to evaluate the user's progress and decide if further actions are needed, or if the task is complete. Although it includes a placeholder tool that performs no action, the Continuation Agent is crucial for maintaining the flow of the conversation, ensuring that the user’s journey continues smoothly based on their prior interactions and the current state. This agent operates with a lower temperature setting to prioritize consistent and logical decision-making.

In [ ]:
# Continuation agent
def continuation_agent_factory(state: dict) -> OpenAIAgent:

    def dummy_tool() -> bool:
        """A tool that does nothing."""
        print("Doing nothing.")

    tools = [
        FunctionTool.from_defaults(fn=dummy_tool)
    ]

    system_prompt = (f"""
        The current state of the user is:
        {pprint.pformat(state, indent=4)}
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini",temperature=0.4),
        system_prompt=system_prompt,
    )


###Orchestration agent
This section defines the Orchestration Agent, which is responsible for determining the next action in a user's interaction with the financial system by selecting the appropriate agent to handle the user's request. The Orchestration Agent evaluates the current state and chat history, using tools to check authentication status and account balance availability. Based on these factors, it returns the name of the next agent to run, ensuring a logical progression through tasks such as stock lookup, authentication, account balance inquiry, or money transfer. The agent operates strictly within predefined rules, providing only the necessary information to guide the flow without adding conversational elements, ensuring a seamless and focused user experience.

In [ ]:

# Orchestration agent
def orchestration_agent_factory(state: dict) -> OpenAIAgent:

    def has_balance() -> bool:
        """Useful for checking if an account has a balance."""
        print("Orchestrator checking if account has a balance")
        return (state["account_balance"] is not None)

    def is_authenticated() -> bool:
        """Checks if the user has a session token."""
        print("Orchestrator is checking if authenticated")
        return (state["session_token"] is not None)

    tools = [
        FunctionTool.from_defaults(fn=has_balance),
        FunctionTool.from_defaults(fn=is_authenticated),
    ]

    system_prompt = (f"""
        You are on orchestration agent.
        Your job is to decide which agent to run based on the current state of the user and what they've asked to do. Agents are identified by short strings.
        What you do is return the name of the agent to run next. You do not do anything else.

        The current state of the user is:
        {pprint.pformat(state, indent=4)}

        If a current_speaker is already selected in the state, simply output that value.

        If there is no current_speaker value, look at the chat history and the current state and you MUST return one of these strings identifying an agent to run:
        * "{Speaker.STOCK_LOOKUP.value}" - if they user wants to look up a stock price (does not require authentication)
        * "{Speaker.AUTHENTICATE.value}" - if the user needs to authenticate
        * "{Speaker.ACCOUNT_BALANCE.value}" - if the user wants to look up an account balance
            * If they want to look up an account balance, but they haven't authenticated yet, return "{Speaker.AUTHENTICATE.value}" instead
        * "{Speaker.TRANSFER_MONEY.value}" - if the user wants to transfer money between accounts (requires authentication and checking an account balance first)
            * If they want to transfer money, but is_authenticated returns false, return "{Speaker.AUTHENTICATE.value}" instead
            * If they want to transfer money, but has_balance returns false, return "{Speaker.ACCOUNT_BALANCE.value}" instead
        * "{Speaker.CONCIERGE.value}" - if the user wants to do something else, or hasn't said what they want to do, or you can't figure out what they want to do. Choose this by default.

        Output one of these strings and ONLY these strings, without quotes.
        NEVER respond with anything other than one of the above five strings. DO NOT be helpful or conversational.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-4o-mini",temperature=0.4),
        system_prompt=system_prompt,
    )


### Agent Initialization and Main Loop
This section outlines the initialization and execution loop of the multi-agent conversational system. The get_initial_state function initializes the user state, setting up key variables such as username, session_token, and account_balance. The run function drives the main interaction loop, guiding the user through various tasks by determining which agent should handle the current request.

It manages the conversation flow, handles user input, and leverages different agents like the Orchestration Agent to decide the next steps. The loop also maintains and updates the conversation history, ensuring that each agent has the context needed to perform its role effectively. This framework allows for dynamic and context-aware interactions within a financial system, ensuring that users receive the appropriate assistance based on their current needs and state.

In [ ]:
# @title Main Loop
def get_initial_state() -> dict:
    return {
        "username": None,
        "session_token": None,
        "account_id": None,
        "account_balance": None,
        "current_speaker": None,
        "just_finished": False,
    }

def run() -> None:
    state = get_initial_state()

    root_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

    first_run = True
    is_retry = False

    while True:
        if first_run:
            # if this is the first run, start the conversation
            user_msg_str = "Hello"
            first_run = False
        elif is_retry == True:
            user_msg_str = "That's not right, try again. Pick one agent."
            is_retry = False
        elif state["just_finished"] == True:
            print("Asking the continuation agent to decide what to do next")
            user_msg_str = str(continuation_agent_factory(state).chat("""
                Look at the chat history to date and figure out what the user was originally trying to do.
                They might have had to do some sub-tasks to complete that task, but what we want is the original thing they started out trying to do.
                Formulate a sentence as if written by the user that asks to continue that task.
                If it seems like the user really completed their task, output "no_further_task" only.
            """, chat_history=current_history))
            print(f"Continuation agent said {user_msg_str}")
            if user_msg_str == "no_further_task":
                user_msg_str = input(">> ").strip()
            state["just_finished"] = False
        else:
            # any other time, get user input
            user_msg_str = input("> ").strip()

        current_history = root_memory.get()

        # who should speak next?
        if (state["current_speaker"]):
            print(f"There's already a speaker: {state['current_speaker']}")
            next_speaker = state["current_speaker"]
        else:
            print("No current speaker, asking orchestration agent to decide")
            orchestration_response = orchestration_agent_factory(state).chat(user_msg_str, chat_history=current_history)
            next_speaker = str(orchestration_response).strip()

        #print(f"Next speaker: {next_speaker}")

        if next_speaker == Speaker.STOCK_LOOKUP:
            print("Stock lookup agent selected")
            current_speaker = stock_lookup_agent_factory(state)
            state["current_speaker"] = next_speaker
        elif next_speaker == Speaker.AUTHENTICATE:
            print("Auth agent selected")
            current_speaker = auth_agent_factory(state)
            state["current_speaker"] = next_speaker
        elif next_speaker == Speaker.ACCOUNT_BALANCE:
            print("Account balance agent selected")
            current_speaker = account_balance_agent_factory(state)
            state["current_speaker"] = next_speaker
        elif next_speaker == Speaker.TRANSFER_MONEY:
            print("Transfer money agent selected")
            current_speaker = transfer_money_agent_factory(state)
            state["current_speaker"] = next_speaker
        elif next_speaker == Speaker.CONCIERGE:
            print("Concierge agent selected")
            current_speaker = concierge_agent_factory(state)
        else:
            print("Orchestration agent failed to return a valid speaker; ask it to try again")
            is_retry = True
            continue

        pretty_state = pprint.pformat(state, indent=4)
        #print(f"State: {pretty_state}")

        # chat with the current speaker
        response = current_speaker.chat(user_msg_str, chat_history=current_history)
        print(Fore.MAGENTA + str(response) + Style.RESET_ALL)

        # update chat history
        new_history = current_speaker.memory.get_all()
        root_memory.set(new_history)

## Run Agents

In [ ]:
if __name__ == "__main__":
    try:
        run()
    except KeyboardInterrupt:
        print("\nExiting gracefully. Goodbye!")


No current speaker, asking orchestration agent to decide
Concierge agent selected

Exiting gracefully. Goodbye!


# Advanced Usages - Adding custom Agents


#### Advanced Usage and Customization:

The system is designed to be highly flexible, allowing for extensive customization and expansion:

- **Adding New Agents**:
  - The architecture supports the easy addition of new agents. Developers can define new agent behaviors, tools, and system prompts, then integrate them into the orchestration logic.
  - Example: You might add an agent for handling investment portfolio analysis, with tools for calculating portfolio performance, assessing risk, and providing recommendations.

- **Customization of Agent Behaviors**:
  - Each agent's behavior can be customized through the use of configurable parameters. For instance, you can adjust the stock lookup agent to fetch real-time data from different financial APIs, or modify the authentication agent to use multi-factor authentication.
  - Configuration UI: Google Colab’s inline UI elements can be used to create interactive forms that allow users to customize agent parameters dynamically before running the notebook.

- **Interactive Configuration**:
  - Users can define and modify agent labels, task descriptions, tool parameters, and even the system prompt directly from the Colab interface, without altering the core codebase.
  - This feature is particularly useful for experimenting with different configurations or adapting the system to different financial environments.

- **Error Handling and Graceful Exit**:
  - The system is built with robust error handling to manage unexpected inputs and user interruptions.
  - For example, the main loop includes a mechanism to gracefully handle user-initiated exits (e.g., via `Ctrl+C`), ensuring that the notebook exits cleanly without displaying unnecessary error messages.


### Adding More Agents to the Notebook

To extend this notebook with additional agents, follow these advanced steps:

#### **Define the New Agent**
   - Create a new function similar to the existing agent factories (e.g., `stock_lookup_agent_factory`, `auth_agent_factory`) for the agent you want to add. The new function should:
     - Define any necessary tools (functions) the agent will use.
     - Set up the system prompt to instruct the agent on how to behave.
     - Return an instance of `OpenAIAgent` configured with the tools and prompt.

#### **Register the New Agent in the `Speaker` Enum**
   - Add an entry for your new agent in the `Speaker` Enum to ensure it is recognized by the orchestration logic.

In [ ]:
# @title Customize Speaker Enum

# Assign text fields for each agent's label
stock_lookup_label = "stock_lookup"  # @param {type:"string"}
authenticate_label = "authenticate"  # @param {type:"string"}
account_balance_label = "account_balance"  # @param {type:"string"}
transfer_money_label = "transfer_money"  # @param {type:"string"}
concierge_label = "concierge"  # @param {type:"string"}
orchestrator_label = "orchestrator"  # @param {type:"string"}
new_agent_label = "new_agent"  # @param {type:"string"}

class Speaker(str, Enum):
    STOCK_LOOKUP = stock_lookup_label.lower()
    AUTHENTICATE = authenticate_label.lower()
    ACCOUNT_BALANCE = account_balance_label.lower()
    TRANSFER_MONEY = transfer_money_label.lower()
    CONCIERGE = concierge_label.lower()
    ORCHESTRATOR = orchestrator_label.lower()
    NEW_AGENT = new_agent_label.lower()


In [ ]:
# @title Configure New Agent

# Assigning values to variables with inline comments for Colab to create UI elements
task_description = "Perform a custom task"  # @param {type:"string"}
tool_param_label = "Parameter for tool"  # @param {type:"string"}
tool_action_response = "Action result"  # @param {type:"string"}
model_name = "gpt-4o-mini"  # @param {type:"string"}
system_prompt_intro = "You are an assistant designed to perform tasks related to"  # @param {type:"string"}

def new_agent_factory(state: dict) -> OpenAIAgent:
    def new_tool_example(param: str) -> str:
        """Description of what this tool does."""
        print(f"Performing action with {param}")
        return f"{tool_action_response} for {param}"

    tools = [
        FunctionTool.from_defaults(fn=new_tool_example),
        # Add more tools if needed
    ]

    system_prompt = (f"""
        {system_prompt_intro} {task_description}.
        The current user state is:
        {pprint.pformat(state, indent=4)}
        Follow the instructions provided to complete your task.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model=model_name),
        system_prompt=system_prompt,
    )


####  **Integrate the New Agent into the Orchestration Logic**
   - Modify the `orchestration_agent_factory` to include logic for your new agent. This involves:
     - Updating the system prompt to account for the new agent’s capabilities.
     - Adding logic to return the identifier for the new agent based on the user’s input and the current state.


In [ ]:
# Define the system prompt for the orchestration agent
system_prompt = (f"""
    You are an orchestration agent. Based on the user's state and input, decide which agent to run next.
    ...
    * "{Speaker.NEW_AGENT.value}" - if the user wants to perform a task related to {Speaker.NEW_AGENT.value}
    ...
""")

# Example of how to set the next speaker using the orchestration agent
if state.get("current_speaker"):
    print(f"There's already a speaker: {state['current_speaker']}")
    next_speaker = state["current_speaker"]
else:
    print("No current speaker, asking orchestration agent to decide")
    current_history = root_memory.get()  # Retrieve the current conversation history
    orchestration_response = orchestration_agent_factory(state).chat(user_msg_str, chat_history=current_history)
    next_speaker = str(orchestration_response).strip()

# Ensure the orchestration logic handles this new agent identifier:
if next_speaker == Speaker.NEW_AGENT:
    print(f"{Speaker.NEW_AGENT.value.capitalize()} agent selected")
    current_speaker = new_agent_factory(state)
    state["current_speaker"] = next_speaker


No current speaker, asking orchestration agent to decide



#### 4. **Test the New Agent**
   - Run the notebook and interact with the new agent to ensure it behaves as expected. Pay particular attention to the agent's interaction with the orchestration logic and other agents.

#### 5. **Advanced Use Cases**
   - **Conditional Logic:** Add more sophisticated logic to the orchestration agent or within individual agents to handle complex decision-making based on multiple state variables.
   - **Memory Sharing:** Use shared memory buffers between agents to maintain context or track long-term goals across multiple user interactions.
   - **Error Handling:** Implement retry mechanisms or fallback logic within agents to gracefully handle unexpected inputs or failed actions.

By following these steps, you can expand the capabilities of your conversational system to handle a broader range of tasks and user interactions. This modular approach ensures that adding new functionality is straightforward and integrates smoothly with the existing architecture.

In [ ]:
if __name__ == "__main__":
    try:
        run()
    except KeyboardInterrupt:
        print("\nExiting gracefully. Goodbye!")

No current speaker, asking orchestration agent to decide

Exiting gracefully. Goodbye!
